In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
from joblib import load, dump


from molmap import dataset
from molmap import loadmap
from molmap import model as molmodel
import molmap

#use GPU, if negative value, CPUs will be used
import tensorflow as tf
#import tensorflow_addons as tfa
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

## fix random seed to get repeatale results
seed = 123
tqdm.pandas(ascii=True)
np.random.seed(seed)
tf.compat.v1.set_random_seed(seed)

RDKit WARNING: [15:03:09] Enabling RDKit 2019.09.2 jupyter extensions


In [2]:
def get_attentiveFP_idx(df):
    """ attentiveFP dataset"""
    train, valid,test = load('../ESOL_train_valid_test.data')
    print('training set: %s, valid set: %s, test set %s' % (len(train), len(valid), len(test)))
    train_idx = df[df.smiles.isin(train.smiles)].index
    valid_idx = df[df.smiles.isin(valid.smiles)].index
    test_idx = df[df.smiles.isin(test.smiles)].index
    print('training set: %s, valid set: %s, test set %s' % (len(train_idx), len(valid_idx), len(test_idx)))
    return train_idx, valid_idx, test_idx 

#load dataset
data = dataset.load_ESOL()
df = data.data
Y = data.y


task_name = 'ESOL'
tmp_feature_dir = './tmpignore'
if not os.path.exists(tmp_feature_dir):
    os.makedirs(tmp_feature_dir)
mp1 = loadmap('../../descriptor.mp')


X1_name = os.path.join(tmp_feature_dir, 'X1_%s.data' % task_name)
if not os.path.exists(X1_name):
    X1 = mp1.batch_transform(df.smiles, n_jobs = 8)
    dump(X1, X1_name)
else:
    X1 = load(X1_name)



train_idx, valid_idx, test_idx = get_attentiveFP_idx(df)
trainY = Y[train_idx]
validY = Y[valid_idx]

total samples: 1128
training set: 901, valid set: 113, test set 113
training set: 901, valid set: 113, test set 113


In [3]:
import time
start_time = str(time.ctime()).replace(':','-').replace(' ','_')
log_file = data.task_name + '_' + start_time + '.log'

with open(log_file,'a') as f:
    f.write(','.join(['n_neighbors', 'min_dist', 'valid_best_rmse', 
                      'train_best_rmse', 'best_epoch'])+'\n')

def Eva(n_neighbors,  min_dist):
    
    min_dist = round(min_dist, 1)    
    n_neighbors = int(n_neighbors)

    print({'min_dist':min_dist, 'n_neighbors':n_neighbors})
    mp_new =  loadmap('../../descriptor.mp')
    mp_new.fit(method = 'umap', min_dist = min_dist, n_neighbors = n_neighbors)
    X_new = mp1.rearrangement(X1, mp_new)
    
    trainX = X_new[train_idx]
    validX = X_new[valid_idx]

    opt = tf.keras.optimizers.Adam(lr = 1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) #    
    model = molmodel.net.SinglePathNet(trainX.shape[1:], 
                                       n_outputs=1, dense_layers=[128, 64], 
                                       dense_avf='tanh', last_avf='linear')
    
    model.compile(optimizer = opt, loss = 'mse')
    performance = molmodel.cbks.Reg_EarlyStoppingAndPerformance((trainX, trainY), 
                                                               (validX, validY), 
                                                               patience = 30, #with out early stopping 
                                                               criteria = 'val_loss')
    model.fit(trainX, trainY, batch_size = 128, 
          epochs=800, verbose= 0, shuffle = True, 
          validation_data = (validX, validY), 
          callbacks=[performance]) 
    
    valid_rmse, valid_r2 = performance.evaluate(validX, validY)
    train_rmse, train_r2 = performance.evaluate(trainX, trainY)
    
    valid_best_rmse = np.nanmean(valid_rmse)
    train_best_rmse = np.nanmean(train_rmse)
    best_epoch = performance.best_epoch
    
    with open(log_file, 'a') as f:
        f.write(','.join([str(min_dist), str(n_neighbors), str(valid_best_rmse), 
                          str(train_best_rmse), str(best_epoch)]) + '\n')
        
    return [valid_best_rmse, train_best_rmse, best_epoch]

In [ ]:
if __name__ == '__main__':
    
    n_neighbors_list = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
    min_dist_list = [0, 0.1, 0.2, 0.3, 0.4,0.5,0.6,0.7, 0.8, 0.9]
    res = []
    for n_neighbors in n_neighbors_list:
        for min_dist in min_dist_list:
            output = Eva(n_neighbors,  min_dist)
            x = [n_neighbors, min_dist]
            x.extend(output)
            res.append(x)

{'min_dist': 0, 'n_neighbors': 10}
UMAP(a=None, angular_rp_forest=False, b=None, init='spectral',
   learning_rate=1.0, local_connectivity=1.0, metric='precomputed',
   metric_kwds=None, min_dist=0, n_components=2, n_epochs=None,
   n_neighbors=10, negative_sample_rate=5, random_state=1,
   repulsion_strength=1.0, set_op_mix_ratio=1.0, spread=1.0,
   target_metric='categorical', target_metric_kwds=None,
   target_n_neighbors=-1, target_weight=0.5, transform_queue_size=4.0,
   transform_seed=42, verbose=2)
Construct fuzzy simplicial set
Wed Jan 22 15:03:17 2020 Finding Nearest Neighbors
Wed Jan 22 15:03:17 2020 Finished Nearest Neighbor Search
Wed Jan 22 15:03:18 2020 Construct embedding


/home/shenwanxiang/.local/lib/python3.6/site-packages/umap/spectral.py:229: UserWarning: Embedding a total of 2 separate connected components using meta-embedding (experimental)
  n_components


	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Wed Jan 22 15:03:21 2020 Finished embedding
2020-01-22 15:03:21,135 - INFO - [bidd-molmap] - Applying grid feature map(assignment), this may take several minutes(1~30 min)
2020-01-22 15:03:23,440 - INFO - [bidd-molmap] - Finished


100%|##########| 1128/1128 [00:00<00:00, 3361.16it/s]


epoch: 0001, loss: 10.8377 - val_loss: 6.8618; rmse: 2.5690 - rmse_val: 2.6195;  r2: 0.3150 - r2_val: 0.2611                                                                                                    
epoch: 0002, loss: 5.3095 - val_loss: 4.1448; rmse: 2.1086 - rmse_val: 2.0359;  r2: 0.3810 - r2_val: 0.3454                                                                                                    
epoch: 0003, loss: 4.6133 - val_loss: 4.3220; rmse: 2.2076 - rmse_val: 2.0789;  r2: 0.4917 - r2_val: 0.4670                                                                                                    
epoch: 0004, loss: 4.7950 - val_loss: 4.1071; rmse: 2.1304 - rmse_val: 2.0266;  r2: 0.5571 - r2_val: 0.5425                                                                                                    
epoch: 0005, loss: 4.4861 - val_loss: 4.1398; rmse: 2.0973 - rmse_val: 2.0347;  r2: 0.5994 - r2_val: 0.5817                                                            

...

epoch: 0001, loss: 9.1635 - val_loss: 5.7287; rmse: 2.3678 - rmse_val: 2.3935;  r2: 0.2445 - r2_val: 0.1819                                                                                                    
epoch: 0002, loss: 4.7416 - val_loss: 4.0774; rmse: 2.1119 - rmse_val: 2.0193;  r2: 0.2367 - r2_val: 0.1816                                                                                                    
epoch: 0003, loss: 4.6618 - val_loss: 4.2293; rmse: 2.1742 - rmse_val: 2.0565;  r2: 0.3012 - r2_val: 0.2494                                                                                                    
epoch: 0004, loss: 4.5606 - val_loss: 3.9833; rmse: 2.0757 - rmse_val: 1.9958;  r2: 0.3746 - r2_val: 0.3200                                                                                                    
epoch: 0005, loss: 4.3018 - val_loss: 4.0642; rmse: 2.0645 - rmse_val: 2.0160;  r2: 0.4227 - r2_val: 0.3660                                                             

/home/shenwanxiang/.local/lib/python3.6/site-packages/umap/spectral.py:229: UserWarning: Embedding a total of 2 separate connected components using meta-embedding (experimental)
  n_components


	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Wed Jan 22 15:57:56 2020 Finished embedding
2020-01-22 15:57:56,736 - INFO - [bidd-molmap] - Applying grid feature map(assignment), this may take several minutes(1~30 min)
2020-01-22 15:58:03,137 - INFO - [bidd-molmap] - Finished


100%|##########| 1128/1128 [00:01<00:00, 864.47it/s]


epoch: 0001, loss: 8.5326 - val_loss: 5.1504; rmse: 2.2614 - rmse_val: 2.2694;  r2: 0.2758 - r2_val: 0.2074                                                                                                    
epoch: 0002, loss: 4.4948 - val_loss: 4.0798; rmse: 2.1237 - rmse_val: 2.0198;  r2: 0.2646 - r2_val: 0.1992                                                                                                    
epoch: 0003, loss: 4.6637 - val_loss: 4.0915; rmse: 2.1327 - rmse_val: 2.0228;  r2: 0.3264 - r2_val: 0.2613                                                                                                    
epoch: 0004, loss: 4.3478 - val_loss: 3.8747; rmse: 2.0331 - rmse_val: 1.9684;  r2: 0.3810 - r2_val: 0.3139                                                                                                    
epoch: 0005, loss: 4.1286 - val_loss: 3.8863; rmse: 2.0150 - rmse_val: 1.9714;  r2: 0.3971 - r2_val: 0.3334                                                             